# Database Test Notebook

In [16]:
%pip install -q mysql-connector-python psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


### References

- https://www.geeksforgeeks.org/how-to-connect-python-with-sql-database/#
- https://www.datacamp.com/tutorial/tutorial-postgresql-python

In [17]:
import os
import mysql.connector
import psycopg2


## Connect to the Database

In [31]:
# Creating connection object
mydb = mysql.connector.connect(
    database = os.environ['DB_NAME'],
    host = os.environ['DB_HOST'],
    user = os.environ['DB_USER'],
    password = os.environ['DB_PASSWORD']
)

In [32]:
# Creating an instance of 'cursor' class which is used to execute the 'SQL' statements in 'Python'
cursor = mydb.cursor()

In [37]:
cursor.execute("SELECT distinct(meta_key) FROM vce_users_meta")

In [ ]:
# Print contents of the cursor
for x in cursor:
  print(x)